In [1]:
%pip install langchain
%pip install langchain_groq
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-8b-instant"
)
# this question went to groq cloud and it asks this question llama-3.3-70b-versatile model and it recieved the response.
response=llm.invoke("The first person to land on moon was ..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.metacareers.com/jobs/1380974339703907/")
# from above URL it extracted all the key info from the HTML structure in the above website and saved in page_data
page_data=loader.load().pop().page_content
print(page_data)



AI Research Scientist, CoreML - Monetization AI | Meta Careers













AI Research Scientist, CoreML - Monetization AISunnyvale, CA • Bellevue, WA • New York, NY • Full Time+ 2 more•Full Time - HideMetaAI ResearchEngineeringApply to this jobWe are the Monetization Ranking AI Research organization, dedicated to delivering personalized ads that maximize both user utility and advertiser value. We focus on advancing AI and ML technologies for all aspects of Monetization, including ranking, retrieval, model architecture, and optimization. By consistently integrating cutting-edge AI/ML advancements, we help Meta’s products achieve long-term goals and have contributed tens of billions in revenue. With our growing impact, we’re seeking AI Research Scientists to join our team and drive SOTA research across the Monetization organization.AI Research Scientist, CoreML - Monetization AI ResponsibilitiesDevelop and implement large-scale model architectures, leveraging model scaling and transf

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract= PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the carrer's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

```json
[
  {
    "role": "AI Research Scientist, CoreML - Monetization AI",
    "experience": "Has obtained a PhD in Computer Science, Computer Engineering, Artificial Intelligence, Machine Learning, or relevant technical field.",
    "skills": [
      "Develop and implement large-scale model architectures, leveraging model scaling and transfer learning techniques",
      "Prioritize training scalability and signal scaling to optimize model performance, efficiency, and reliability",
      "Develop and apply NextGen sequence learning techniques to drive advancements in natural language processing and understanding",
      "Design and implement generative modeling solutions for data augmentation",
      "Research and develop graph-aware large language models",
      "Develop and deploy AutoML pipelines",
      "Apply Reinforcement Learning (RL) techniques, including long-term value optimization, RLHF, and RL4Reason",
      "Use causal learning to identify and understand the cause and ef

str

In [9]:
# To convert the text into JSON object we use JSON parser

from langchain_core.output_parsers import JsonOutputParser


json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'AI Research Scientist, CoreML - Monetization AI',
  'experience': 'Has obtained a PhD in Computer Science, Computer Engineering, Artificial Intelligence, Machine Learning, or relevant technical field.',
  'skills': ['Develop and implement large-scale model architectures, leveraging model scaling and transfer learning techniques',
   'Prioritize training scalability and signal scaling to optimize model performance, efficiency, and reliability',
   'Develop and apply NextGen sequence learning techniques to drive advancements in natural language processing and understanding',
   'Design and implement generative modeling solutions for data augmentation',
   'Research and develop graph-aware large language models',
   'Develop and deploy AutoML pipelines',
   'Apply Reinforcement Learning (RL) techniques, including long-term value optimization, RLHF, and RL4Reason',
   'Use causal learning to identify and understand the cause and effect of relationships across data',
   'Collabor

In [10]:
type(json_res)

list

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
links =  collection.query(query_texts=["Experience in Python", "Expertise in Data Science"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [16]:
job=json_res
job

[{'role': 'AI Research Scientist, CoreML - Monetization AI',
  'experience': 'Has obtained a PhD in Computer Science, Computer Engineering, Artificial Intelligence, Machine Learning, or relevant technical field.',
  'skills': ['Develop and implement large-scale model architectures, leveraging model scaling and transfer learning techniques',
   'Prioritize training scalability and signal scaling to optimize model performance, efficiency, and reliability',
   'Develop and apply NextGen sequence learning techniques to drive advancements in natural language processing and understanding',
   'Design and implement generative modeling solutions for data augmentation',
   'Research and develop graph-aware large language models',
   'Develop and deploy AutoML pipelines',
   'Apply Reinforcement Learning (RL) techniques, including long-term value optimization, RLHF, and RL4Reason',
   'Use causal learning to identify and understand the cause and effect of relationships across data',
   'Collabor

In [29]:
links =  collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-tv-portfoli

In [35]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
     ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Vamsi Adi, a Software Engineer at Infosys Ltd. Infosys Ltd is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, I have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Vamsi        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Vamsi's portfolio: {link_list}
        Remember you are Vamsi Adi, Software Engineer  at Infosys Ltd. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)

In [36]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI Solutions for CoreML - Monetization AI

Dear Hiring Manager,

I came across the job description for an AI Research Scientist, CoreML - Monetization AI, and I am excited to introduce myself as a strong candidate who can fulfill your requirements. As a Software Engineer at Infosys Ltd, I have a proven track record of empowering enterprises with tailored AI and software solutions, driving scalability, process optimization, cost reduction, and heightened overall efficiency.

With a strong foundation in computer science and software engineering, I possess the necessary skills to develop and implement large-scale model architectures, prioritize training scalability, and optimize model performance, efficiency, and reliability. My expertise in Python, PyTorch, and machine learning techniques, including model scaling and transfer learning, next-generation sequence learning, generative modeling, and reinforcement learning, aligns perfectly with your job requirements.

I would 